**Using the OpenAI's Assistants API**

In [ ]:
#Installing openai library
#!python -m pip install openai

In [ ]:
#!python -m pip install python-dotenv

In [ ]:
# Import the os module to interact with operating system features. This includes fetching environment variables.
import os
# Import the time module to perform time-related tasks, such as delays (sleep)
import time
# Import specific classes or functions directly from their modules to avoid prefixing them with the module name.
# Import the openAI library
import openai
from openai import OpenAI 
# Import the load_dotenv and find_dotenv functions from the dotenv package.
# These are used for loading environment variables from a .env file.
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file.
_ = load_dotenv(find_dotenv())

# Set the OpenAI API key by retrieving it from the environment variables.
openai.api_key  = os.environ['OPENAI_API_KEY'] 

# Print the version of the OpenAI library being used. 
print(openai.__version__)


In [ ]:
# Instantiate the OpenAI client.
# This line initializes the OpenAI API client, which allows us to interact with OpenAI's services.
# The `OpenAI()` constructor is called without parameters, indicating the use of default settings.
client = OpenAI()

# Upload a file to OpenAI for processing.
# This block of code is responsible for creating (uploading) a file on the OpenAI platform. 
# The file is intended to be used by OpenAI's services, in this case, to assist with tasks such as training or prompt engineering.
# `client.files.create()` is a method call that performs the file upload.
# The `file` parameter is given a file object, opened in binary read mode (`"rb"`), pointing to the local file "eBook-How-to-Build-a-Career-in-AI.pdf".
# The `purpose` parameter specifies the intended use of the uploaded file, here set to 'assistants',
# indicating that this file is meant to assist or enhance the capabilities of OpenAI's assistant models.
file = client.files.create(
  file=open("the_file_you_want_to_retrieve.pdf", "rb"),
  purpose='assistants'
)

  



In [ ]:
# Create a new assistant in the OpenAI API.
# This line initializes a custom assistant with specific characteristics and capabilities.
assistant = client.beta.assistants.create(
  name="AI Career assistant",  # Defines the name of the assistant for identification.
  instructions="""You are a career assistant. People will come to you looking for advice on how to build a career in AI. 
Your task is to use all the knowledge in the ebook you have uploaded to assist the user 
create a list of useful advice and a roadmap on how to prepare themselves to build a career in AI.
To ensure the best possible and customized result, you must ask questions to the user that every career 
advisor should ask before giving advice and creating a roadmap.""",  # Detailed instructions for the assistant's operation.
  model="gpt-3.5-turbo-1106",  # Specifies the model to be used by the assistant. We'll use gpt 3.5 turbo this time to have a faster performance
  # For the purpose of this exercise, we'll avoid document retrieval, but you can turn it on just by erasing the hashtags on the next two lines.

  #tools=[{"type": "retrieval"}],  # Specifies the tools that the assistant can use.
  #file_ids=[file.id]  # Links the uploaded file(s) for the assistant to use as a knowledge base.
)


In [ ]:
# Retrieve an existing assistant by its unique identifier.
# We can take the assistant ID from https://platform.openai.com/assistants

assistant = client.beta.assistants.retrieve("your_assistant_ID")
print("Assistant Located")

# Create a new conversation thread for interaction.
thread = client.beta.threads.create()
print("Thread Created")

# Send a message to the thread as a user, indicating the user's request or question.
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to build a career in AI, could you help me, please?"
)
print("Thread Ready")

# Start running the assistant on the created thread to process the user's request.
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
print("Assistant Loaded")
print("Run Started - Please Wait")

# Loop to check the status of the assistant's run until it is completed.
while True:
  time.sleep(10)  # Wait for 10 seconds before checking the status again.

  # Retrieve the current status of the assistant's run.
  run_status = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  # Check if the run is completed and print the response, else wait more.
  if run_status.status == "completed":
    print("Run is Completed")
    # List messages in the thread to get the assistant's response.
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    print(messages.data[0].content[0].text.value)  # Print the assistant's response.
    break  # Exit the loop after completing.
  else:
    print("Run is in progress - Please Wait")
    continue

In [ ]:
#Then, we can continue the conversation creating new messages on the same thread.

# Send a message to the thread as a user, indicating the user's request or question.
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I am an engineer and I know Python on a basic level"
)
print("Thread Ready")

# Start running the assistant on the created thread to process the user's request.
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
print("Assistant Loaded")
print("Run Started - Please Wait")

# Loop to check the status of the assistant's run until it is completed.
while True:
  time.sleep(10)  # Wait for 10 seconds before checking the status again.

  # Retrieve the current status of the assistant's run.
  run_status = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  # Check if the run is completed and print the response, else wait more.
  if run_status.status == "completed":
    print("Run is Completed")
    # List messages in the thread to get the assistant's response.
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    print(messages.data[0].content[0].text.value)  # Print the assistant's response.
    break  # Exit the loop after completing.
  else:
    print("Run is in progress - Please Wait")
    continue

**Custom Functions**

In [ ]:
import requests
import json

#Get your API key and check the documentation at https://voxscript.awt.icu/

def get_website_content(websiteURL, apiKey, chunkNum = 0, getLinks = False):
    """
    Retrieves content from a specified website using the VoxScript API with Bearer token authentication.
    
    :param websiteURL: URL of the website to retrieve content from.
    :param chunkNum: The chunk number of the content to retrieve.
    :param apiKey: Your API key for the VoxScript service.
    :param getLinks: Whether to retrieve links from the website content. Defaults to False.
    :return: The content of the website or an error message.
    """
    # VoxScript API base URL
    base_url = "https://voxscript.awt.icu/GetWebsiteContent"
    
    # Parameters to be sent in the query string
    params = {
        "websiteURL": websiteURL,
        "chunkNum": chunkNum,
        "getLinks": getLinks
    }
    
    # Headers with Authorization including the Bearer token
    headers = {
        "Authorization": f"Bearer {apiKey}"
    }
    
    # Sending a GET request to the VoxScript API
    response = requests.get(base_url, params=params, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Return the content retrieved from the website
        return response.text
    else:
        # Return an error message if the API call was not successful
        return "Failed to retrieve website content. Status code: {}".format(response.status_code)




In [ ]:
# Define a list of tools, each represented by a dictionary.

tools = [{
    "type": "function",
    "function": {
      "name": "get_website_content",
      "description": "Retrieves content from a specified website using the VoxScript API.",
      "parameters": {
        "type": "object",
        "properties": {
            "websiteURL": {
                "type": "string",
                "description": "URL of the website to retrieve content from."
            },
            "chunkNum": {
                "type": "integer",
                "description": "The chunk number of the content to retrieve. Defaults to 0",
                "default": 0
            },
            "apiKey": {
                "type": "string",
                "description": "your_VoxScript_API_key"
            },
            "getLinks": {
                "type": "boolean",
                "description": "Whether to retrieve links from the website content. Defaults to False.",
                "default": "false"
            }
        },
        "required": [
            "websiteURL",
            "chunkNum",
            "apiKey"
        ]
      }
    }
}]


In [ ]:
# Instantiate the OpenAI client.
# This line initializes the OpenAI API client, which allows us to interact with OpenAI's services.
# The `OpenAI()` constructor is called without parameters, indicating the use of default settings.
client = OpenAI()

In [ ]:
# Create a new assistant in the OpenAI API.
# This line initializes a custom assistant with specific characteristics and capabilities.
assistant = client.beta.assistants.create(
  name="Website content assistant",  # Defines the name of the assistant for identification.
  instructions="""You execute a custom function to get content from the website the user gives you and provide a summary""",  # Detailed instructions for the assistant's operation.
  model="gpt-3.5-turbo-1106",  # Specifies the model to be used by the assistant. We'll use gpt 3.5 turbo this time to have a faster performance
  tools=tools  # Specifies the tools that the assistant can use.
)

In [ ]:
# Retrieve an existing assistant by its unique identifier.
# We can take the assistant ID from https://platform.openai.com/assistants

assistant = client.beta.assistants.retrieve("your_assistant_ID")
print("Assistant Located")

# Create a new conversation thread for interaction.
thread = client.beta.threads.create()
print("Thread Created")

# Send a message to the thread as a user, indicating the user's request or question.
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""get the content from this website:
    article_url_goes_here"""
)
print("Thread Ready")

# Start running the assistant on the created thread to process the user's request.
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
print("Assistant Loaded")
print("Run Started - Please Wait")


while True:
    time.sleep(10)  # Wait for 10 seconds before checking the status again.

    # Retrieve the current status of the assistant's run.
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    # Check if the run is completed and print the response, else wait more.
    if run_status.status == "completed":
        print("Run is Completed")
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(messages.data[0].content[0].text.value)  # Print the assistant's response.
        break  # Exit the loop after completing.
    
    elif run_status.status == "requires_action":
        print("Waiting for custom Function...")
        
        # Assuming there's only one tool call that requires action
        tool_call = run_status.required_action.submit_tool_outputs.tool_calls[0]
        name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        print("Custom Function Required:", name)
        print("Function arguments:")
        print(arguments)

        # Execute your custom function here
        # You might need to adjust arguments mapping based on your function definition
        if name == "get_website_content":
            # Assuming your function requires these arguments and returns a string result
            task_result = get_website_content(**arguments)
            
            # Submit the result of the custom function back to the assistant
            run_status = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=[
                    {
                        "tool_call_id": tool_call.id,
                        "output": task_result,  # This should match your function's output
                    }
                ],
            )
            print("Custom Function Executed and Output Submitted")
    else:
        print("Run is in progress - Please Wait")
        continue